In [20]:
import numpy as np
import pandas as pd
import os

# -----------------------------
# Constants / parameters
# -----------------------------
KT_TO_MS = 0.514444
NM_TO_M  = 1852.0
R_EARTH  = 6371000.0  # m

RHO_A = 1.225
RHO_W = 1025.0
G     = 9.81
CD    = 1.5e-3

BETA_TRANSLATION = 1.0   # translation weighting
ALPHA_OUTER = 1.0        # Rankine outer decay exponent
CROSS_TOL_KM = 0.5       # treat |dist - RMW| <= tol as a crossing "touch"


# -----------------------------
# Helpers: angles, geodesy
# -----------------------------
def angle_diff_deg(a, b):
    return (a - b + 180.0) % 360.0 - 180.0

def haversine_m(lat1, lon1, lat2, lon2):
    lat1 = np.deg2rad(lat1); lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2); lon2 = np.deg2rad(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2.0 * R_EARTH * np.arcsin(np.sqrt(a))

def bearing_deg(lat1, lon1, lat2, lon2):
    lat1 = np.deg2rad(lat1); lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2); lon2 = np.deg2rad(lon2)
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(dlon)
    brng = np.rad2deg(np.arctan2(x, y))
    return (brng + 360.0) % 360.0

def heading_to_unit_EN(heading_deg):
    th = np.deg2rad(heading_deg)
    e = np.sin(th)
    n = np.cos(th)
    return e, n

def EN_to_heading_deg(e, n):
    hdg = np.rad2deg(np.arctan2(e, n))
    return (hdg + 360.0) % 360.0


# -----------------------------
# Read sites CSV
# -----------------------------
def read_sites(sites_csv, colmap=None):
    if colmap is None:
        colmap = dict(site_id="site_id", lat="lat", lon="lon", L_m="L_m", h_m="h_m", az_deg="az_deg")

    df = pd.read_csv(sites_csv)
    df.columns = df.columns.str.strip()

    missing = [colmap[k] for k in colmap if colmap[k] not in df.columns]
    if missing:
        raise ValueError(f"Sites CSV missing columns: {missing}")

    return {
        "site_id": df[colmap["site_id"]].astype(str).values,
        "lat":     df[colmap["lat"]].astype(float).values,
        "lon":     df[colmap["lon"]].astype(float).values,
        "L_m":     df[colmap["L_m"]].astype(float).values,
        "h_m":     df[colmap["h_m"]].astype(float).values,
        "az_deg":  df[colmap["az_deg"]].astype(float).values
    }


# -----------------------------
# Rankine + translation wind model
# -----------------------------
def rankine_speed(Vmax_ms, r_m, rmw_m, alpha_outer=1.0):
    if (rmw_m is None) or (rmw_m <= 0) or (r_m is None) or (r_m <= 0) or np.isnan(rmw_m) or np.isnan(r_m):
        return np.nan
    if r_m <= rmw_m:
        return Vmax_ms * (r_m / rmw_m)
    else:
        return Vmax_ms * (rmw_m / r_m)**alpha_outer

def wind_vector_rankine_plus_translation(
    latc, lonc, lat_site, lon_site,
    Vmax_ms, rmw_m,
    trans_ms, trans_heading_deg,
    beta_translation=1.0,
    alpha_outer=1.0,
    hemisphere="NH"
):
    r_m = haversine_m(latc, lonc, np.array([lat_site]), np.array([lon_site]))[0]
    br  = bearing_deg(latc, lonc, lat_site, lon_site)  # center -> site

    e_r, n_r = heading_to_unit_EN(br)

    # tangential unit: NH CCW => rotate +90°
    if hemisphere.upper() == "NH":
        e_t, n_t = (-n_r, e_r)
    else:
        e_t, n_t = (n_r, -e_r)

    V_tan = rankine_speed(Vmax_ms, r_m, rmw_m, alpha_outer=alpha_outer)

    e_tr, n_tr = heading_to_unit_EN(trans_heading_deg)
    u_tr = beta_translation * trans_ms * e_tr
    v_tr = beta_translation * trans_ms * n_tr

    u = V_tan * e_t + u_tr
    v = V_tan * n_t + v_tr

    U10 = np.hypot(u, v)
    dir_to = EN_to_heading_deg(u, v)
    dir_from = (dir_to + 180.0) % 360.0

    return U10, dir_from, dir_to, r_m


# -----------------------------
# Main computation
# -----------------------------
def compute_rmw_crossing_events(
    ibtracs_csv,
    sites_csv,
    out_csv,
    sites_colmap=None,
    verbose=True
):
    sites = read_sites(sites_csv, colmap=sites_colmap)
    site_id = sites["site_id"]
    lat_s   = sites["lat"]
    lon_s   = sites["lon"]
    L_s     = sites["L_m"]
    h_s     = sites["h_m"]
    az_s    = sites["az_deg"]

    want_cols = [
        "SID","NAME","SEASON","BASIN","ISO_TIME","LAT","LON",
        "USA_WIND","USA_RMW",
        "USA_R34_NE","USA_R34_SE","USA_R34_SW","USA_R34_NW",
        "USA_R50_NE","USA_R50_SE","USA_R50_SW","USA_R50_NW",
    ]

    df = pd.read_csv(ibtracs_csv, skiprows=[1], low_memory=False)
    df.columns = df.columns.str.strip()

    keep = [c for c in want_cols if c in df.columns]
    if not all(c in keep for c in ["SID","ISO_TIME","LAT","LON","USA_RMW"]):
        raise ValueError("Need at least SID, ISO_TIME, LAT, LON, USA_RMW in IBTrACS CSV.")

    df = df[keep].copy()

    for c in ["SID","NAME","BASIN"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().replace({"nan": np.nan})

    df["ISO_TIME"] = pd.to_datetime(df["ISO_TIME"], errors="coerce")
    df = df.dropna(subset=["SID","ISO_TIME","LAT","LON","USA_RMW"])

    num_cols = [c for c in df.columns if c not in ["SID","NAME","BASIN"]]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    for c in df.columns:
        if c.startswith("USA_") or c in ["LAT","LON"]:
            if pd.api.types.is_numeric_dtype(df[c]):
                df.loc[df[c].isin([-9999, -999, -99]), c] = np.nan

    df = df.sort_values(["SID","ISO_TIME"]).reset_index(drop=True)

    if verbose:
        print("Rows after dropping missing SID/time/center/RMW:", len(df))
        print("Non-null USA_RMW:", int(df["USA_RMW"].notna().sum()))

    os.makedirs(os.path.dirname(out_csv) or ".", exist_ok=True)
    out_cols = [
        "site_id","site_lat","site_lon","L_m","h_m","az_deg",
        "storm_sid","storm_name","season","basin",
        "cross_time_utc",
        "U10_mps","wind_dir_from_deg","wind_dir_to_deg",
        "cos2_weight","eta_setup_m",
        "dist_km","rmw_km","vmax_kt",
        "within_R34","within_R50",
        "R34_nm_max","R50_nm_max",
        "storm_lat","storm_lon",
        "trans_mps","trans_heading_deg"
    ]
    pd.DataFrame(columns=out_cols).to_csv(out_csv, index=False)

    def interp_rmax(g, i, f, thresh):
        cols = [f"USA_R{thresh}_NE", f"USA_R{thresh}_SE", f"USA_R{thresh}_SW", f"USA_R{thresh}_NW"]
        if not all(c in g.columns for c in cols):
            return np.nan
    
        v_i = g[cols].iloc[i].to_numpy(float)
        v_j = g[cols].iloc[i+1].to_numpy(float)
    
        # Treat 0 as missing if present
        v_i = np.where(v_i == 0, np.nan, v_i)
        v_j = np.where(v_j == 0, np.nan, v_j)
    
        r_i = np.nan if not np.isfinite(v_i).any() else float(np.nanmax(v_i))
        r_j = np.nan if not np.isfinite(v_j).any() else float(np.nanmax(v_j))
    
        if np.isfinite(r_i) and np.isfinite(r_j):
            return float(r_i + f * (r_j - r_i))
        elif np.isfinite(r_i):
            return float(r_i)
        elif np.isfinite(r_j):
            return float(r_j)
        else:
            return np.nan


    n_events = 0

    for sid, g in df.groupby("SID", sort=False):
        g = g.sort_values("ISO_TIME")
        if len(g) < 2:
            continue

        # Force datetime64[ns] so dt math is reliable
        t = g["ISO_TIME"].to_numpy(dtype="datetime64[ns]")
        latc = g["LAT"].to_numpy(float)
        lonc = g["LON"].to_numpy(float)

        vmax_kt = g["USA_WIND"].to_numpy(float) if "USA_WIND" in g.columns else np.full(len(g), np.nan)
        rmw_nm  = g["USA_RMW"].to_numpy(float)

        name = g["NAME"].iloc[0] if "NAME" in g.columns else ""
        season = g["SEASON"].iloc[0] if "SEASON" in g.columns else np.nan
        basin  = g["BASIN"].iloc[0]  if "BASIN"  in g.columns else np.nan

        rmw_km = (rmw_nm * NM_TO_M) / 1000.0

        for j in range(len(site_id)):
            d_m  = haversine_m(latc, lonc, np.full(len(latc), lat_s[j]), np.full(len(lonc), lon_s[j]))
            d_km = d_m / 1000.0

            delta = d_km - rmw_km

            finite_pair = np.isfinite(delta[:-1]) & np.isfinite(delta[1:])
            cross_prod  = finite_pair & ((delta[:-1] * delta[1:]) < 0)
            touch_i   = np.isfinite(delta[:-1]) & (np.abs(delta[:-1]) <= CROSS_TOL_KM)
            touch_ip1 = np.isfinite(delta[1:])  & (np.abs(delta[1:])  <= CROSS_TOL_KM)

            cross_idx = np.where(cross_prod | touch_i | touch_ip1)[0]
            if len(cross_idx) == 0:
                continue

            for i in cross_idx:
                di = delta[i]
                dj2 = delta[i+1]

                if np.isfinite(di) and np.isfinite(dj2) and (di * dj2 < 0) and (dj2 - di) != 0:
                    f = float(np.clip(-di / (dj2 - di), 0.0, 1.0))
                else:
                    if np.isfinite(di) and np.isfinite(dj2):
                        f = 0.0 if abs(di) <= abs(dj2) else 1.0
                    elif np.isfinite(di):
                        f = 0.0
                    else:
                        f = 1.0

                ti = t[i]
                tj = t[i+1]
                cross_time = ti + (tj - ti) * np.float64(f)

                lat_x = latc[i] + f * (latc[i+1] - latc[i])
                lon_x = lonc[i] + f * (lonc[i+1] - lonc[i])

                vmax_x_kt = vmax_kt[i] + f * (vmax_kt[i+1] - vmax_kt[i]) if np.isfinite(vmax_kt[i]) and np.isfinite(vmax_kt[i+1]) else np.nan
                rmw_x_km  = rmw_km[i]  + f * (rmw_km[i+1]  - rmw_km[i])  if np.isfinite(rmw_km[i])  and np.isfinite(rmw_km[i+1])  else np.nan
                dist_x_km = float(d_km[i] + f * (d_km[i+1] - d_km[i]))

                # dt in seconds (robust): use ns integer math
                dt_ns = (tj - ti).astype("timedelta64[ns]").astype(np.int64)
                dt_s = dt_ns / 1e9

                if np.isfinite(dt_s) and dt_s > 0:
                    seg_dist_m = haversine_m(latc[i], lonc[i], np.array([latc[i+1]]), np.array([lonc[i+1]]))[0]
                    trans_ms = float(seg_dist_m / dt_s)
                    trans_hdg = float(bearing_deg(latc[i], lonc[i], latc[i+1], lonc[i+1]))
                else:
                    trans_ms = 0.0
                    trans_hdg = 0.0

                Vmax_ms = vmax_x_kt * KT_TO_MS if np.isfinite(vmax_x_kt) else np.nan
                rmw_x_m = rmw_x_km * 1000.0 if np.isfinite(rmw_x_km) else np.nan

                U10, dir_from, dir_to, _ = wind_vector_rankine_plus_translation(
                    lat_x, lon_x, float(lat_s[j]), float(lon_s[j]),
                    Vmax_ms, rmw_x_m,
                    trans_ms, trans_hdg,
                    beta_translation=BETA_TRANSLATION,
                    alpha_outer=ALPHA_OUTER,
                    hemisphere="NH"
                )

                dtheta = angle_diff_deg(dir_to, float(az_s[j]))
                cos2 = float(np.cos(np.deg2rad(dtheta))**2)

                tau_eff = RHO_A * CD * (U10**2) * cos2
                eta = (tau_eff * float(L_s[j])) / (RHO_W * G * float(h_s[j]))

                R34_nm = interp_rmax(g, i, f, 34)
                R50_nm = interp_rmax(g, i, f, 50)
                within_R34 = bool(np.isfinite(R34_nm) and (dist_x_km * 1000.0 <= R34_nm * NM_TO_M))
                within_R50 = bool(np.isfinite(R50_nm) and (dist_x_km * 1000.0 <= R50_nm * NM_TO_M))

                row = {
                    "site_id": site_id[j],
                    "site_lat": float(lat_s[j]),
                    "site_lon": float(lon_s[j]),
                    "L_m": float(L_s[j]),
                    "h_m": float(h_s[j]),
                    "az_deg": float(az_s[j]),
                    "storm_sid": sid,
                    "storm_name": str(name),
                    "season": season,
                    "basin": basin,
                    "cross_time_utc": pd.to_datetime(cross_time, utc=True),
                    "U10_mps": float(U10),
                    "wind_dir_from_deg": float(dir_from),
                    "wind_dir_to_deg": float(dir_to),
                    "cos2_weight": cos2,
                    "eta_setup_m": float(eta),
                    "dist_km": float(dist_x_km),
                    "rmw_km": float(rmw_x_km) if np.isfinite(rmw_x_km) else np.nan,
                    "vmax_kt": float(vmax_x_kt) if np.isfinite(vmax_x_kt) else np.nan,
                    "within_R34": within_R34,
                    "within_R50": within_R50,
                    "R34_nm_max": float(R34_nm) if np.isfinite(R34_nm) else np.nan,
                    "R50_nm_max": float(R50_nm) if np.isfinite(R50_nm) else np.nan,
                    "storm_lat": float(lat_x),
                    "storm_lon": float(lon_x),
                    "trans_mps": float(trans_ms),
                    "trans_heading_deg": float(trans_hdg),
                }

                pd.DataFrame([row]).to_csv(out_csv, mode="a", index=False, header=False)
                n_events += 1

    if verbose:
        print(f"Wrote: {out_csv}")
        print(f"Total crossing events written: {n_events}")

    return out_csv

In [22]:
ib_csv  = "ibtracs.NA.list.v04r01.csv"
sites   = "lagoon_metrics.csv"
outcsv  = "sites_rmw_crossings_setup_events_rankine_cos2_flags.csv"

compute_rmw_crossing_events(ib_csv, sites, outcsv, verbose=True)

Rows after dropping missing SID/time/center/RMW: 127649
Non-null USA_RMW: 21951
Wrote: sites_rmw_crossings_setup_events_rankine_cos2_flags.csv
Total crossing events written: 124


,site_id,site_lat,site_lon,L_m,h_m,az_deg,storm_sid,storm_name,season,basin,...,rmw_km,vmax_kt,within_R34,within_R50,R34_nm_max,R50_nm_max,storm_lat,storm_lon,trans_mps,trans_heading_deg
0,EMatagorda,28.7039,-95.8407,28000.0,3.0,65.0,2001157N28265,ALLISON,2001,NaN,...,92.600000,43.526108,True,False,156.471078,NaN,29.384222,-95.300000,4.118331,0.000000
1,Pamlico,35.2340,-76.0540,100000.0,4.0,53.0,2002196N34283,ARTHUR,2002,NaN,...,74.080000,30.000000,False,False,NaN,NaN,34.617303,-76.086068,8.676212,61.408035
2,Pamlico,35.2340,-76.0540,100000.0,4.0,53.0,2002196N34283,ARTHUR,2002,NaN,...,74.080000,30.000000,False,False,NaN,NaN,34.832544,-75.546550,7.431666,65.213563
3,Matagorda,28.2370,-96.7330,75000.0,3.0,231.0,2002249N28266,FAY,2002,NaN,...,74.106007,50.000000,True,False,82.394680,NaN,28.350702,-96.038729,7.564902,303.149069
4,Matagorda,28.2370,-96.7330,75000.0,3.0,231.0,2002249N28266,FAY,2002,NaN,...,90.733935,32.878842,True,False,60.000000,NaN,29.013635,-96.813635,8.213743,318.915839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,EMatagorda,28.7039,-95.8407,28000.0,3.0,65.0,2024181N09320,BERYL,2024,NaN,...,53.506050,72.495349,True,True,107.504651,70.0,29.133643,-96.000000,6.177496,0.000000
120,Pamlico,35.2340,-76.0540,100000.0,4.0,53.0,2024216N20284,DEBBY,2024,NaN,...,360.639589,30.000000,False,False,NaN,NaN,36.089192,-79.903603,6.399271,15.070094
121,Pamlico,35.2340,-76.0540,100000.0,4.0,53.0,2024216N20284,DEBBY,2024,NaN,...,388.920000,30.000000,False,False,NaN,NaN,37.118150,-79.677847,16.653100,8.335644
122,Albemarle,36.0570,-76.0950,75000.0,3.0,88.0,2024216N20284,DEBBY,2024,NaN,...,350.963992,30.000000,False,False,NaN,NaN,35.880216,-79.973261,6.399271,15.070094


In [23]:
import pandas as pd

# --- paths ---
events_csv = "sites_rmw_crossings_setup_events_rankine_cos2_flags.csv"  # <-- change if needed

# --- read + ensure numeric types where appropriate ---
df = pd.read_csv(events_csv, parse_dates=["cross_time_utc"])

num_cols = ["U10_mps","wind_dir_from_deg","eta_setup_m","dist_km","rmw_km","vmax_kt","R34_nm_max","R50_nm_max"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# If within_R34/within_R50 came in as strings ("True"/"False"), normalize to bool
for c in ["within_R34","within_R50"]:
    if c in df.columns and df[c].dtype == object:
        df[c] = df[c].astype(str).str.strip().str.lower().map({"true": True, "false": False})

# --- select columns + rank by eta (descending) ---
cols = [
    "site_id","storm_sid","storm_name","cross_time_utc",
    "U10_mps","wind_dir_from_deg","eta_setup_m","dist_km","rmw_km","vmax_kt",
    "within_R34","within_R50","R34_nm_max","R50_nm_max"
]

ranked_storms = (
    df.assign(eta_setup_m=pd.to_numeric(df["eta_setup_m"], errors="coerce"))
      .sort_values("eta_setup_m", ascending=False)
      .groupby(["site_id","storm_sid"], as_index=False)
      .first()  # keeps the row with max eta because we pre-sorted
      .loc[:, cols]
      .sort_values("eta_setup_m", ascending=False)
      .reset_index(drop=True)
)

ranked_storms.head(50)


,site_id,storm_sid,storm_name,cross_time_utc,U10_mps,wind_dir_from_deg,eta_setup_m,dist_km,rmw_km,vmax_kt,within_R34,within_R50,R34_nm_max,R50_nm_max
0,PadreIs,2020205N26272,HANNA,2020-07-25 20:09:05.951043461+00:00,40.913038,344.031349,8.248513,37.040000,37.040000,80.000000,True,True,100.000000,50.000000
1,Pamlico,2003249N14329,ISABEL,2003-09-18 15:40:44.811769187+00:00,49.771380,91.425147,6.945828,76.526280,76.526280,90.000000,True,False,300.000000,NaN
2,Albemarle,2018280N18273,MICHAEL,2018-10-12 04:24:06.283234554+00:00,37.482083,271.416382,6.395535,127.132956,127.132956,57.336242,True,False,240.000000,68.646304
3,Pamlico,2018280N18273,MICHAEL,2018-10-12 01:21:53.785349979+00:00,35.901396,241.571913,5.757581,192.748188,192.748188,52.274901,True,False,240.000000,104.075696
4,Pamlico,2013157N25273,ANDREA,2013-06-07 23:12:41.495930485+00:00,33.969049,240.883753,5.172406,222.240000,222.240000,40.000000,True,False,200.000000,NaN
5,Albemarle,2011233N15301,IRENE,2011-08-28 00:32:24.732863123+00:00,33.616796,284.439646,4.749321,92.677599,92.677599,65.000000,True,True,230.041900,150.000000
6,SPadreIs,2008203N18276,DOLLY,2008-07-23 14:48:13.698616137+00:00,40.839440,22.741781,4.723946,37.040000,37.040000,83.392390,True,True,120.000000,48.392390
7,Pamlico,2011233N15301,IRENE,2011-08-27 20:20:21.019021804+00:00,35.852649,259.356917,4.714954,83.340000,83.340000,65.000000,True,True,225.000000,135.136412
8,Albemarle,2013157N25273,ANDREA,2013-06-08 01:27:34.369457698+00:00,30.665875,254.042476,4.046259,222.240000,222.240000,40.000000,True,False,200.000000,NaN
9,Albemarle,2021182N09317,ELSA,2021-07-09 01:34:04.642151357+00:00,33.018125,233.774896,3.404993,159.904339,159.904339,46.567956,True,False,140.000000,NaN
